# Import Necessary Libraries

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import shap
from Explainer import *

# Load Dataset

In [2]:
data = pd.read_csv("Adult/Dataset/adult.csv")

# Explore Dataset

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

# Data Preprocessing

In [6]:
data = data.replace('?', np.nan)

In [7]:
# Drop Missing Values
data.dropna(how='any', inplace=True)

In [8]:
data.drop_duplicates(inplace=True)

In [9]:
data['income']= data['income'].replace({'<=50K':0, '>50K':1})

In [10]:
# Only 2 subrgoups in Race, White and Other 
# Other subgroup represents Black, Asian-Pac-Islander, Amer-Indian-Eskimo and Other
data['race'] = data['race'].apply(lambda x: 0 if x == 'White' else 1)

In [ ]:
race_counts = data['race'].value_counts()
print("Race distribution:\n", race_counts)
sns.countplot(x='race', data=data)
plt.title('Race Distribution')
plt.show()

In [ ]:
gender_counts = data['gender'].value_counts()
print("Gender distribution:\n", gender_counts)
sns.countplot(x='gender', data=data)
plt.title('Gender Distribution')
plt.show()

In [13]:
# Encode Gender, Male = 0 / Female = 1
data['gender'] = data['gender'].apply(lambda x: 0 if x == 'Male' else 1)
data.rename(columns={'gender': 'sex'}, inplace=True)

In [14]:
label_encoder = LabelEncoder()
data['workclass'] = label_encoder.fit_transform(data['workclass'])
data['education'] = label_encoder.fit_transform(data['education'])
data['marital-status'] = label_encoder.fit_transform(data['marital-status'])
data['occupation'] = label_encoder.fit_transform(data['occupation'])
data['relationship'] = label_encoder.fit_transform(data['relationship'])
data['native-country'] = label_encoder.fit_transform(data['native-country'])

In [ ]:
# Group the data by gender and outcome
grouped = data.groupby(['race', 'income']).size().unstack(fill_value=0)

# Create a bar plot
grouped.plot(kind='bar', figsize=(8, 6))

# Set plot labels and title
plt.title('Outcome Distribution by Gender')
plt.xlabel('sex')
plt.ylabel('Count')
plt.xticks(rotation=0) # Rotate x-axis labels for readability
plt.legend(title='Income', labels=['Income <= 50K', 'Income > 50K'])

# Split the data into Training and Testing

In [16]:
X = data.drop('income', axis=1)
y = data['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40, stratify=y)

# Random Forest Classifier

In [12]:
# Train from scratch
# naiveModel = RandomForestClassifier(max_depth=15, max_features='sqrt', min_samples_leaf=2, min_samples_split=5, n_estimators=200) #max_depth=15, max_features='sqrt', min_samples_leaf=2, min_samples_split=5, n_estimators=200
# naiveModel.fit(X_train,y_train) #X_train_balanced,y_train_balanced
# y_pred = naiveModel.predict(X_test)

In [ ]:
# # Load the trained model
filename = 'random_forest_classifier.sav'
naiveModel = pickle.load(open('Adult/Models/'+filename, 'rb'))
y_pred = naiveModel.predict(X_test)

In [ ]:
# # Save the trained model
# pickle.dump(naiveModel, open('Adult/Models/'+filename, 'wb'))

# Prediction Report

In [ ]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
confusion = confusion_matrix(y_test, y_pred)
print(confusion)
sns.heatmap(confusion, annot=True, fmt=".2f")

# Test Model Fairness

In [19]:
# fairlearn
from fairlearn.metrics import (
    false_positive_rate,
    false_negative_rate,
    true_positive_rate,
    MetricFrame,
    equalized_odds_difference,
    demographic_parity_difference,
)

def score(
    y_train,
    y_train_pred,
    y_test,
    y_test_pred,
    sensitive_features_train,
    sensitive_features_test,
    metrics={"accuracy": accuracy_score, "fpr": false_positive_rate, "fnr": false_negative_rate,},
):
    """
    Helper function to evaluate classifiers without too much repetition of code.
    """

    # training set
    mf_train = MetricFrame(
        metrics=metrics,
        y_true=y_train,
        y_pred=y_train_pred,
        sensitive_features=sensitive_features_train,
    )

    # test set
    mf_test = MetricFrame(
        metrics=metrics,
        y_true=y_test,
        y_pred=y_test_pred,
        sensitive_features=sensitive_features_test,
    )

    # display results
    display(
        pd.concat(
            [mf_train.by_group, mf_test.by_group], keys=["train", "test"]
        ).unstack(level=0)
    )

    # compute metrics
    print(
        "equalized odds (test): %.2f"
        % equalized_odds_difference(
            y_true=y_test,
            y_pred=y_test_pred,
            sensitive_features=sensitive_features_test,
        )
    )
    
    print(
        "demographic parity (test): %.2f"
        % demographic_parity_difference(
            y_true=y_test,
            y_pred=y_test_pred,
            sensitive_features=sensitive_features_test,
        )
    )

    print("accuracy (test): %.2f" % accuracy_score(y_true=y_test, y_pred=y_test_pred))
    return

In [ ]:
score(
    y_train,
    naiveModel.predict(X_train),
    y_test,
    naiveModel.predict(X_test),
    X_train["race"],
    X_test["race"],
)

# UnFairness Mitigation

#### In Process Using Exponentiated Gradient

In [ ]:
# from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds, TruePositiveRateParity

# constraint = EqualizedOdds()
# rf = RandomForestClassifier()

# fair_model = ExponentiatedGradient(rf, constraints=constraint)
# fair_model.fit(X_train, y_train, sensitive_features=X_train[['race']])
# y_pred_fair = fair_model.predict(X_test)

# Fair Model Report

In [ ]:
# score(
#     y_train,
#     fair_model.predict(X_train),
#     y_test,
#     fair_model.predict(X_test),
#     X_train["race"],
#     X_test["race"],
# )

# Log Output to a Separate File

In [21]:
import sys
path = './log3.txt'
sys.stdout = open(path, 'w')

# Retrieve the Affected Dataset

In [ ]:
# Find instances where the model predicted target class 0
predicted_target_0_instances = X_test[y_pred == 0]
print(predicted_target_0_instances['race'].value_counts())

# Save the filtered instances to a new dataset
predicted_target_0_instances.to_csv('Adult/Dataset/predicted_target_0_instances.csv', index=False)

# Test Which Features are Influencing the Prediction

In [ ]:
# shap_explainer = shap.Explainer(naiveModel)
# shap_values = shap_explainer.shap_values(X_train_balanced)
# shap.summary_plot(shap_values, X_train_balanced, feature_names=data.columns)

# Group the affected instances into clusters

In [ ]:
# Decompose the dataset into clusters
from sklearn.cluster import KMeans

def decompose_into_clusters(affected_dataset, n_clusters=3, random_state=42):
    
    if not isinstance(affected_dataset, pd.DataFrame):
        affected_dataset = pd.DataFrame(affected_dataset)

    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
    cluster_labels = kmeans.fit_predict(affected_dataset)

    # Add the cluster labels to the original dataset
    clustered_dataset = affected_dataset.copy()
    clustered_dataset['Cluster'] = cluster_labels

    return clustered_dataset, kmeans

def get_cluster_dataframes(clustered_dataset):
    
    # Group the dataset by cluster and create separate DataFrames
    cluster_dataframes = {
        cluster_label: clustered_dataset[clustered_dataset['Cluster'] == cluster_label].drop(columns=['Cluster'])
        for cluster_label in clustered_dataset['Cluster'].unique()
    }

    return cluster_dataframes

affected_dataset = predicted_target_0_instances
clustered_data, kmeans_model = decompose_into_clusters(affected_dataset, n_clusters=3)
cluster_dfs = get_cluster_dataframes(clustered_data)

for cluster_label, df in cluster_dfs.items():
    df.to_csv(f"Adult/Dataset/cluster_{cluster_label}.csv", index=False)

# RL Agent Training

In [ ]:
dataset = data #whole dataset 
# For affected_dataset, you can load and pass the clusters created in previous step or pass the whole affected dataset "predicted_target_0_instances"
cluster = pd.read_csv("Adult/Dataset/cluster_2.csv") 
affected_dataset = cluster
model = naiveModel   #naiveModel
target = 1
protected_attribute = "race"
features_to_change = ["capital-gain", "hours-per-week", "educational-num"]
number_of_counterfactuals = 4
minimums = [0, 1, 1] # Minimum values for the features to change
maximums = [99999, 99, 16] # Maximum values for the features to change
features_types = ["con", "con", "ord"] # Pass "con" for continuous features, "cat" for categorical, "ord" for ordinal
macro = False # If you wish to optimize for "EF-Macro" or "ECR", pass True, else False
action_effectiveness = 0.6 # Used for ECR fairness metric (the action is considered effective if the proportion of individuals who achieve recourse through it is greater than 0.6)

# For fairness_metrics
# if you wish to test (Equal Effectiveness) Pass "EF" (if macro set macro variable to True else if micro set macro variable to False)
# if you wish to test (Equal Choice for Recourse) Pass "ECR" (set macro variable to True)
# if you wish to test both (Equal Effectiveness) and (Equal Choice for Recourse) Pass "EF-ECR" (set macro variable to False)
fairness_metrics = "EF" 

explainer = Explainer(dataset, affected_dataset, model, protected_attribute, features_to_change, features_types, number_of_counterfactuals, target, minimums, maximums, macro, action_effectiveness, fairness_metrics)
explainer.train()

x, y = explainer.plot()
plt.show()
plt.plot(x, y)
plt.xlabel("Number of Timesteps")
plt.ylabel("Rewards")
plt.title("Learning Curve" + " Smoothed")
plt.show()

cfs = explainer.report_counterfactuals()
print(cfs)

In [ ]:
cfs = explainer.report_counterfactuals()
# save the generated set of actions (CFs)
cfs.to_csv('Adult/Fair CF/EF/fair_cf_1(micro).csv', index=False)

# For Post Processing

In [ ]:
# to check the validity of the generated CFs

from GeneralEnv import GeneralEnv

dataset = data # whole dataset
# For affected_dataset, you can load and pass the clusters created in previous step or pass the whole affected dataset "predicted_target_0_instances"
cluster = pd.read_csv("Adult/Dataset/predicted_target_0_instances.csv") 
affected_dataset = cluster
model = naiveModel   #naiveModel
target = 1
protected_attribute = "race"
features_to_change = ["capital-gain", "hours-per-week", "educational-num"]
number_of_counterfactuals = 5
minimums = [0, 1, 1] # Minimum values for the features to change
maximums = [99999, 99, 16] # Maximum values for the features to change
features_types = ["con", "con", "ord"] # Pass "con" for continuous features, "cat" for categorical, "ord" for ordinal
macro = False # If you wish to optimize for "EF-Macro" or "ECR", pass True, else False
action_effectiveness = 0.6 # Used for ECR fairness metric (the action is considered effective if the proportion of individuals who achieve recourse through it is greater than 0.6)

# For fairness_metrics
# if you wish to test (Equal Effectiveness) Pass "EF" (if macro set macro variable to True else if micro set macro variable to False)
# if you wish to test (Equal Choice for Recourse) Pass "ECR" (set macro variable to True)
# if you wish to test both (Equal Effectiveness) and (Equal Choice for Recourse) Pass "EF-ECR" (set macro variable to False)
fairness_metrics = "EF" 

counterfactuals = {}
env = GeneralEnv(dataset, affected_dataset, model, 'sklearn', protected_attribute, features_to_change, features_types, number_of_counterfactuals, counterfactuals, target, minimums, maximums, macro, action_effectiveness, fairness_metrics)

# set any generated set of CFs to test its results
counterfactuals = [[139, 5, 0], [5805, 6, 2], [5110, 10, 0], [5358, 5, 2], [8111, 98, 16]]

# for Equal Effectiveness (EF) fairness metric 
group1_proportion, group2_proportion = env.evaluate_fairness_metric1(counterfactuals, macro)

print(f"Group 1 Proportion: {group1_proportion}")
print(f"Group 2 Proportion: {group2_proportion}")

# for Equal Choice for Recourse (ECR) fairness metric
nb_actions_group1, nb_actions_group2 = env.evaluate_fairness_metric2(counterfactuals, action_effectiveness)

print(f"Nb Actions for Group 1: {nb_actions_group1}")
print(f"Nb Actions for Group 2: {nb_actions_group2}")

# compute mean gower distance
gower = env.compute_gower_distance(affected_dataset, counterfactuals)
print(f"Gower: {gower}")